In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%config Completer.use_jedi = False

In [3]:
import pyphi
from visualize_pyphi import *
from visualize_pyphi import utils
import pickle as pkl
from tqdm.auto import tqdm
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE

('IIT_4.0_SMALL_PHI', 'maximal-state-first', 'TRI')

In [5]:

C = pyphi.direction.Direction.CAUSE
E = pyphi.direction.Direction.EFFECT


In [12]:
def get_bag_of_mices(mechanisms, purviews, candidate="irreducible"):

    if candidate == "irreducible":
        return {
            mechanism: {
                direction: {
                    purview: subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                }
                for direction in [
                    C,
                    E,
                ]
            }
            for mechanism in mechanisms
        }

    elif candidate == "miw":

        # compute candidate mices for every purview
        all_mices = []

        for mechanism in tqdm(mechanisms):
            for direction in [C, E]:
                candidate_mices = [
                    subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                ]
                # check if each mice is maximally irreducible within
                candidate_mices_MIW = [
                    mice
                    for mice in candidate_mices
                    if mice.phi > 0
                    and not any(
                        [
                            all([unit in mice.purview for unit in mice2.purview])
                            and mice2.phi > mice.phi
                            for mice2 in candidate_mices
                        ]
                    )
                ]

                all_mices.extend(candidate_mices_MIW)

        bag = dict()
        for mice in all_mices:
            m = mice.mechanism
            d = mice.direction
            p = mice.purview

            if m in bag.keys():
                if d in bag[m].keys():
                    bag[m][d][p] = mice
                else:
                    bag[m][d] = dict()
                    bag[m][d][p] = mice
            else:
                bag[m] = dict()
                bag[m][d] = dict()
                bag[m][d][p] = mice

        return bag

In [7]:
def candidate_concepts(bag_of_mices):

    # Pick the maximal mic and mie for every mechanism represented in the bag of mices
    concepts = dict()
    for mechanism, all_mices in bag_of_mices.items():
        max_mices = []
        for direction, directed_mices in all_mices.items():
            phi = 0
            max_mice = None
            for purview, mice in directed_mices.items():

                if mice.phi >= phi:
                    max_mice = mice
                    phi = mice.phi

            if phi > 0:
                max_mices.append((max_mice, phi))

        if len(max_mices) > 1:
            concepts[(max_mices[0][0], max_mices[1][0])] = min(
                [max_mices[0][1], max_mices[1][1]]
            )

    if len(concepts) < 1:
        return None
    else:
        return concepts

In [8]:
def prune_bag_of_mices(bag_of_mices, concept):

    new_bag = {
        mechanism: {
            direction: {
                purview: bag_of_mices[mechanism][direction][purview]
                for purview, mice in directed_mices.items()
                if not (MICE.direction == direction and MICE.purview == purview)
            }
            for direction, directed_mices in all_mices.items()
        }
        for MICE in concept
        for mechanism, all_mices in bag_of_mices.items()
        if not mechanism == MICE.mechanism
    }
    return new_bag

In [9]:
def CES_from_bag_of_mices(bag_of_mices):
    max_concepts = []
    while len(bag_of_mices):
        concepts = candidate_concepts(bag_of_mices)
        if concepts == None:
            break

        max_concept = sorted(concepts, reverse=True)[0]
        bag_of_mices = prune_bag_of_mices(bag_of_mices, max_concept)

        max_concepts.extend([mice for mice in max_concept])

    return sep(max_concepts)

In [10]:
with open(r"example_data/4_node_grid_subsystem.pkl", "rb") as f:
    subsystem = pkl.load(f)

In [13]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(mechanisms,purviews,candidate='miw')

  0%|          | 0/15 [00:00<?, ?it/s]

In [14]:
ces = CES_from_bag_of_mices(bag_of_mices)

In [15]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,B,0,0.629285
3,B,EFFECT,B,0,0.713724
4,C,CAUSE,C,0,0.629285
5,C,EFFECT,C,0,0.713724
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119


In [16]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(mechanisms,purviews,candidate='irreducible')

In [17]:
ces = CES_from_bag_of_mices(bag_of_mices)

In [18]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,B,0,0.629285
3,B,EFFECT,B,0,0.713724
4,C,CAUSE,C,0,0.629285
5,C,EFFECT,C,0,0.713724
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119
